In [ ]:
# mounted drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
import numpy as np
import os

folder_path = '/content/drive/MyDrive/Master/Multimedia/fruit'


# Xây dựng mô hình Autoencoder
def build_autoencoder(input_shape=(299, 299, 3), encoding_dim=100):
    # Encoder
    input_img = Input(shape=input_shape)
    flattened = Flatten()(input_img)
    encoded = Dense(encoding_dim, activation='relu')(flattened)

    # Decoder
    decoded = Dense(np.prod(input_shape), activation='sigmoid')(encoded)
    decoded_reshaped = Reshape(input_shape)(decoded)

    # Mô hình Autoencoder
    autoencoder = Model(input_img, decoded_reshaped)
    encoder = Model(input_img, encoded)

    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    return autoencoder, encoder

# Tải và chuẩn bị hình ảnh
def load_and_preprocess_images(folder_path, target_size=(299, 299)):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        if os.path.isfile(img_path) and (filename.endswith(".jpg") or filename.endswith(".png")):
            img = image.load_img(img_path, target_size=target_size)
            img_array = image.img_to_array(img)
            img_array = img_array / 255.0  # Chuẩn hóa về [0, 1]
            images.append(img_array)
    return np.array(images)

# Địa chỉ thư mục chứa hình ảnh
data_x = load_and_preprocess_images(folder_path)

# Xây dựng và huấn luyện mô hình
autoencoder, encoder = build_autoencoder()
autoencoder.fit(data_x, data_x, epochs=50, batch_size=256, shuffle=True, validation_split=0.2)

# Hàm giảm chiều dữ liệu bằng Encoder
def reduce_dimensionality_with_encoder(img_path, encoder):
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img) / 255.0  # Chuẩn hóa dữ liệu ảnh
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)

    # Sử dụng encoder để giảm chiều
    reduced_features = encoder.predict(img_array_expanded_dims)
    return reduced_features.flatten()

Epoch 1/50
1/1 [==============================] - 6s 6s/step - loss: 0.6932 - val_loss: 0.6932
Epoch 2/50
1/1 [==============================] - 0s 410ms/step - loss: 0.6932 - val_loss: 0.6923
Epoch 3/50
1/1 [==============================] - 1s 525ms/step - loss: 0.6930 - val_loss: 0.6931
Epoch 4/50
1/1 [==============================] - 0s 452ms/step - loss: 0.6931 - val_loss: 0.6928
Epoch 5/50
1/1 [==============================] - 0s 457ms/step - loss: 0.6931 - val_loss: 0.6909
Epoch 6/50
1/1 [==============================] - 0s 469ms/step - loss: 0.6911 - val_loss: 0.6827
Epoch 7/50
1/1 [==============================] - 0s 400ms/step - loss: 0.6845 - val_loss: 0.6684
Epoch 8/50
1/1 [==============================] - 0s 414ms/step - loss: 0.6741 - val_loss: 0.6577
Epoch 9/50
1/1 [==============================] - 0s 417ms/step - loss: 0.6676 - val_loss: 0.6455
Epoch 10/50
1/1 [==============================] - 0s 410ms/step - loss: 0.6570 - val_loss: 0.6364
Epoch 11/50
1/1 [=====

In [ ]:
import json

def save_features_to_json(folder_path, encoder, file_name='features.json'):
    features_list = []
    # Duyệt qua tất cả các tệp trong thư mục
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        if os.path.isfile(img_path) and (filename.endswith(".jpg") or filename.endswith(".png")):
            # Sử dụng hàm reduce_dimensionality_with_encoder để lấy vector đặc trưng giảm chiều
            reduced_features = reduce_dimensionality_with_encoder(img_path, encoder)
            features_list.append({'path': img_path, 'features': reduced_features.tolist()})

    # Lưu vào tệp JSON
    with open(file_name, 'w') as f:
        json.dump(features_list, f, indent=4)

# Gọi hàm để lưu vector
save_features_to_json(folder_path, encoder)

1/1 [==============================] - 0s 19ms/step


In [ ]:
# Lưu toàn bộ mô hình Autoencoder
autoencoder.save('/content/drive/MyDrive/Master/Multimedia/autoencoder_model.h5')

# Hoặc, nếu bạn chỉ muốn sử dụng phần Encoder
# encoder.save('/content/drive/MyDrive/Master/Multimedia/encoder_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
